In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline, BitsAndBytesConfig
from peft import PeftModel
import torch
from trl import setup_chat_format
import pandas as pd

In [2]:
model_name = "IlyaGusev/saiga_llama3_8b"
new_model = "saiga_llama3_8b-ruFinGPT"

In [3]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
)

base_model_reload = AutoModelForCausalLM.from_pretrained(
        model_name,
        return_dict=True,
        low_cpu_mem_usage=True,
        torch_dtype=torch.float16,
        device_map="auto",
        trust_remote_code=True,
        quantization_config=bnb_config,
)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [4]:
base_model_reload, tokenizer = setup_chat_format(base_model_reload, tokenizer)

# Merge adapter with base model
model = PeftModel.from_pretrained(base_model_reload, new_model)

model = model.merge_and_unload()

C:\Users\Sergey\.conda\envs\edok\Lib\site-packages\peft\tuners\lora\bnb.py:336: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(


In [5]:
df = pd.read_csv('dataset/fines_ru.csv')[:5]
def format_chat_template(row):
    row = row[1]
    row_json = [
        {"role": "system", "content": row["instruction"]},
        {"role": "user", "content": row["input"]}         
    ]
    res_text = "<|begin_of_text|>"
    for item in row_json:
        res_text += f"<|start_header_id|>{item['role']}<|end_header_id|>{item['content']}<|eot_id|>"
        
    return res_text

df['text'] = [format_chat_template(row) for row in df.iterrows()]
df['text'][2]

'<|begin_of_text|><|start_header_id|>system<|end_header_id|>Какая тональность у этой новости? Пожалуйста, выберите вариант ответа из {сильно негативно/умеренно негативно/слабо негативно/нейтрально/слабо позитивно/умеренно позитивно/сильно позитивно}.<|eot_id|><|start_header_id|>user<|end_header_id|>Шрёдер отклонил предложение войти в совет директоров «Газпрома»<|eot_id|>'

In [9]:
for i, item in df.iterrows():
    prompt = item["text"]
    
    pipe = pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        torch_dtype=torch.float16,
        device_map="auto",
    )
    
    outputs = pipe(
        prompt, 
        max_new_tokens=128, 
        do_sample=True, 
        temperature=0.5, 
        top_k=1, 
        top_p=0.8, 
        repetition_penalty=1.2, 
        no_repeat_ngram_size=2,
        length_penalty=0.8
    )
    print(outputs[0]["generated_text"])
    print("-----------------------------------------")

<|begin_of_text|><|start_header_id|>system<|end_header_id|>Какая тональность у этой новости? Пожалуйста, выберите вариант ответа из {сильно негативно/умеренно негативно/слабо негативно/нейтрально/слабо позитивно/умеренно позитивно/сильно позитивно}.<|eot_id|><|start_header_id|>user<|end_header_id|>Электромобильный стартап Arrival экс-главы Yota уйдет из России<|eot_id|>﻿#Yota #Arrival #РоссияHeaderCode: 2022-04-22T14:00:47Zystemуменительно негативное системаassistantслабо негативная системные ошибкиsystemумеро положительное systemумералятельно позитивное в будущем..systemсладко-позитивное на долгое время. В настоящее время компания работает над расширением своего присутствия и развития новых рынков для электоральных автомобилей. Следующее поколение едва ли не
-----------------------------------------
<|begin_of_text|><|start_header_id|>system<|end_header_id|>Какая тональность у этой новости? Пожалуйста, выберите вариант ответа из {сильно негативно/умеренно негативно/слабо негативно/ней